# Native Finetune Diffusion 集成训练环境
一种更native的stable diffusion模型finetune方式 (改进于原始的dreambooth)

本环境是对日本大佬@Kohya S方法的在线运行封装
https://note.com/kohya_ss/n/nbf7ce8d80f29  

主要包含以下特性：
- 支持finetune unet
- 支持finetune text encoder
- 支持更长的clip token输入长度限制（75， 150， 225）
- 支持采用clip倒数第二层的输出（以更接近NovelAI的训练技巧）
- 支持使用非固定分辨率训练，以保留更多图像信息（无需先裁剪数据集为512x512）
- 支持图像的自动标注（prompt / tags）
- 支持训练Stable diffusion 2.0 

## 前置环境处理
把缓存文件归位(节省后面下载的时间)

In [ ]:
!rm -rf /root/.cache
!mv /root/models-cache /root/.cache

## 进行数据标注

这里推荐手动标注数据（对于AI难以进行自动标注的数据，比如工业零件、纹身、图案、水墨画、国风等），或使用下面的脚本工具创建自动标注。


### 一、针对真实图像数据（生成自然描述）
生成自然语言描述，速度相对比较慢

In [ ]:
!python tools/label_images_with_caption.py ./datasets/test

合并caption到json列表

In [ ]:
!python merge_captions_to_metadata.py \
        ./datasets/test \
        meta_cap.json # 输出的tags标注列表

### 二、针对二次元数据（生成danbooru tags）
生成由大量标签组成的描述(tags)

In [ ]:
!python tools/label_images_with_tags.py \
        --path ./datasets/test \
        --threshold 0.75 # 阈值

合并tags标签到json

这里也可通过指定--in_json meta_cap.json把上面带有caption的标注放进来，追加tags标记，训练时将会同时读取caption和tags一块进行训练。

In [ ]:
!python merge_dd_tags_to_metadata.py \
        ./datasets/test \
        --in_json meta_cap.json \
        meta_cap_dd.json # 输出的tags标注列表

### 三、清洗标注列表
- 如果只进行了自然描述的标注生成，请把下方meta_cap_dd.json改为meta_cap.json

In [ ]:
!python clean_captions_and_tags.py \
        ./datasets/test \
        --in_json meta_cap_dd.json \
        --out_json meta_clean.json # 清洗后输出的标注列表

## 预处理训练样本为隐变量
- --max_resolution默认512,512
- 如训练sd二代模型，请添加 --v2  


In [ ]:
!python prepare_buckets_latents.py \
    ./datasets/test  \
    meta_clean.json \
    meta_lat.json \
    --batch_size 4 \
    --mixed_precision fp16 \
    --max_resolution 768,768 \
    ./models/novel_lastest.ckpt # 待训练的基础模型

## Native Finetune 训练

### 训练SD 1代模型：
- --pretrained_model_name_or_path 基底模型路径  
- --train_data_dir 数据集路径  
- --save_precision=fp16 以半精度保存权重
- --save_every_n_epochs 每训练多少个epoch保存一次ckpt检查点（注意磁盘空间容量，如果满了会导致训练失败）
- --max_token_length 225、115、75可选
- --clip_skip=2 使用clip倒数第二层的输出来进行训练，更接近NovelAI的方式（需要使用时在webui同样设置clip_skip=2）

In [ ]:
!rm -rf /root/tf-logs/*
!accelerate launch --num_cpu_threads_per_process 8 fine_tune.py \
    --pretrained_model_name_or_path=./models/novel_lastest.ckpt \
    --in_json meta_lat.json \
    --train_data_dir=./datasets/test \
    --output_dir=./fine_tuned \
    --shuffle_caption \
    --logging_dir="/root/tf-logs" \
    --train_batch_size=1 --learning_rate=5e-6 --max_train_steps=10000 \
    --use_8bit_adam --xformers --gradient_checkpointing \
    --mixed_precision=fp16 \
    --max_token_length=225 \
    --clip_skip=2 \
    --save_every_n_epochs=100

### 训练SD二代模型

- 训练768x768模型时，需要指定--v_parameterization (512x512的Stable diffusion-base2.0模型不需要) 

769-v-ema.ckpt可通过以下命令下载：  
wget https://huggingface.co/stabilityai/stable-diffusion-2/resolve/main/768-v-ema.ckpt

In [ ]:
!rm -rf /root/tf-logs/*
!accelerate launch --num_cpu_threads_per_process 8 fine_tune.py \
    --pretrained_model_name_or_path=./models/768-v-ema.ckpt \
    --in_json meta_lat.json \
    --train_data_dir=./datasets/test \
    --output_dir=./fine_tuned \
    --shuffle_caption \
    --logging_dir="/root/tf-logs" \
    --train_batch_size=1 --learning_rate=5e-6 --max_train_steps=10000 \
    --use_8bit_adam --xformers --gradient_checkpointing \
    --mixed_precision=fp16 \
    --save_precision=fp16 \
    --save_every_n_epochs=100 \
    --v2 \
    --v_parameterization 

具体说明可以查看原作者的教程（日文）：  
https://note.com/kohya_ss/n/nbf7ce8d80f29

对应视频教程可在主页查找，https://space.bilibili.com/291593914

如果感兴趣，可以加XDiffusion交流群 455521885 

## 其他

为了方便使用，避免频繁在多个代码间切换的麻烦，在tools目录里也集成了多个自己常用的工具脚本，涵盖模型融合、图像批处理、原始dreambooth、Textual inversion、prune、上传cos等，可自行研究取用。

如：模型融合
```
!python tools/ckpt_merge.py A.ckpt B.ckpt --alpha 0.3 --without_vae
```